In [1]:
import pandas as pd
import numpy as np

In [2]:
sensor_name = 'R1'
corr_xx = pd.read_feather(f'../data/corr/XX/corr_XX-{sensor_name}.feather')
corr_xy = pd.read_feather(f'../data/corr/XY/corr_XY-{sensor_name}.feather')

In [3]:
def IFS(corXX, corXY, level_xx, level_xy):
    corXY = corXY.copy().values
    mask = [False] * len(corXX)
    while np.max(corXY) > level_xy:
        i_bestXY = np.argmax(corXY)
        # print("Лучший:", i_bestXY, "\nПохожие:")
        mask[i_bestXY] = True
        corXY[i_bestXY] = 0
        for i in range(corXX.shape[0]):
            if mask[i] == False:
                if corXY[i]>0:
                    if corXX.iloc[i_bestXY, i]>level_xx:
                        corXY[i] = 0
                        # print(i)
    return mask

In [ ]:
level_xx = 0.995
level_xy = 0.0
elem_names = [i for i in corr_xy.columns]
masks = []
for element in elem_names:
    ifs_result = IFS(corr_xx, corr_xy[element], level_xx, level_xy)
    masks.append([element] + ifs_result)

In [8]:
from config import masks_path
masks_path.mkdir(parents=True, exist_ok=True)
pd.DataFrame(np.array(masks).T[1:], columns=np.array(masks).T[0]).to_feather(masks_path / 'masks.feather')

In [7]:
# num_features = [np.sum(IFS(corr_xx, corr_xy['NO_conc'], i/10000, 0.0)) for i in range(9500,9999,10)]
# import matplotlib.pyplot as plt
# plt.plot(num_features)
# num_features[44]
# result = list(zip(num_features, [i/10000 for i in range(9500,9999,10)]))
# result[45]